In [1]:
import struct
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt,pi

In [2]:
def print_result(posterior,ans):
    print('Postirior (in log scale):')
    for i in range(10):
        print(i,': ',posterior[i])
    print('Prediction: {}, Ans: {}\n'.format(np.argmin(posterior),int(ans)))

In [3]:
def print_imagination_continuous(mean):
    print('Imagination of numbers in Bayesian classifier:\n')
    for i in range(10):
        print(i,':')
        for p in range(28*28):
            if mean[i][p]<128:
                print('0', end=' ')
            else:
                print('1', end=' ')
            if p%28==27:
                print('')
        print('')

In [4]:
def print_imagination(train_result):
    print('Imagination of numbers in Bayesian classifier:\n')
    for i in range(10):
        print(i,':')
        for p in range(28*28):
            no=yes=0
            for j in range(16):
                if(train_result[i][p][j]!=0):
                    no+=train_result[i][p][j]
                else:
                    no+=0.0001/train_label_count[i]
            for j in range(16,32):
                if(train_result[i][p][j]!=0):
                    yes+=train_result[i][p][j]
                else:
                    yes+=0.0001/train_label_count[i]
            if no>yes:
                print('0', end=' ')
            else:
                print('1', end=' ')
            if p%28==27:
                print('')
        print('')

In [5]:
def test_continuous(mean,varience,N):
    error=0
    for i in range(N):
        posterior=np.copy(prior)
        for l in range(10):
            for p in range(28*28):
                posterior[l]+=np.log(1/np.sqrt(2*pi*varience[l][p]))-(((test_image[i][p]-mean[l][p])**2)/(2*varience[l][p]))-np.log(train_label_count[l])
        #posterior/=np.sum(posterior)
        #print_result(posterior,test_label[i])
        if(np.argmax(posterior)!=test_label[i]):
            error+=1
    #print_imagination_continuous(mean)
    #print('Error rate: ',float(error/N))
    return float(error/N)

In [6]:
def train_continuous(N,var):
    mean=np.zeros((10,28*28))
    varience=np.zeros((10,28*28))
    for i in range(N):
        for p in range(28*28):
            mean[int(train_label[i])][p]+=train_image[i][p]
            varience[int(train_label[i])][p]+=train_image[i][p]**2
    for l in range(10):
        mean[l]=mean[l]/train_label_count[l]
        varience[l]/=train_label_count[l]
        varience[l]-=mean[l]**2
        for p in range(28*28):
            if(varience[l][p]==0):
                varience[l][p]=var**2
    return mean, varience

In [7]:
def test_discrete(train_result,N):
    error=0
    for i in range(N):
        posterior=np.copy(prior)
        for label in range(10):
            for p in range(28*28):
                if(train_result[label][p][int(test_image[i][p]//8)]!=0):
                    posterior[label]+=np.log(train_result[label][p][int(test_image[i][p]//8)])
                else:
                    posterior[label]+=np.log(0.0001/train_label_count[label])
        posterior=posterior/np.sum(posterior)
        print_result(posterior,test_label[i])
        if(np.argmin(posterior)!=test_label[i]):
            error+=1
    print_imagination(train_result)
    print('Error rate: ',float(error/N))

In [8]:
def train_discrete(N):
    image_bin_count=np.zeros((10,28*28,32))
    probability=np.zeros((10,28*28,32))
    for i in range(N):
        if(i%10000==0):
            print(i,' images has trained.')
        for p in range(28*28):
            image_bin_count[int(train_label[i])][p][int(train_image[i][p]//8)]+=1
    for label in range(10):
        probability[label]=image_bin_count[label]/train_label_count[label]
    return probability

In [9]:
def idx3_decode(file):
    data=open(file, 'rb').read()
    # > for Big-endian, iiii for 4 integers, each size=4
    fmt='>iiii'
    offset=0
    magic_number, image_numbers, height, width=struct.unpack_from(fmt,data,offset)
    image_size=height*width
    offset+=struct.calcsize(fmt)
    # B for unsigned byte, size=1
    fmt='>'+str(image_size)+'B'
    images=np.empty((image_numbers,height*width))
    for i in range(image_numbers):
        images[i]=np.array(struct.unpack_from(fmt,data,offset)).reshape((height*width))
        offset+=struct.calcsize(fmt)
    return images,image_numbers

In [10]:
def idx1_decode(file):
    data=open(file, 'rb').read()
    # > for Big-endian, ii for 2 integers, each size=4
    fmt='>ii'
    offset=0
    magic_number, label_numbers=struct.unpack_from(fmt,data,offset)
    offset+=struct.calcsize(fmt)
    # B for unsigned byte, size=1
    fmt='>B'
    labels=np.empty(label_numbers)
    for i in range(label_numbers):
        labels[i]=struct.unpack_from(fmt,data,offset)[0]
        offset+=struct.calcsize(fmt)
    return labels,label_numbers

In [11]:
train_image='train-images.idx3-ubyte'
train_label='train-labels.idx1-ubyte'
test_image='t10k-images.idx3-ubyte'
test_label='t10k-labels.idx1-ubyte'

In [12]:
train_image,train_image_number=idx3_decode(train_image)
test_image,test_image_number=idx3_decode(test_image)
train_label,train_label_number=idx1_decode(train_label)
test_label,test_label_number=idx1_decode(test_label)

In [13]:
train_label_count=np.zeros(10)
for label in train_label:
    train_label_count[int(label)]+=1
train_label_count

array([5923., 6742., 5958., 6131., 5842., 5421., 5918., 6265., 5851.,
       5949.])

In [14]:
prior=np.log(train_label_count/train_label_number)
prior

array([-2.31550148, -2.18598795, -2.30960971, -2.28098669, -2.32927136,
       -2.40406426, -2.31634601, -2.25936597, -2.32773198, -2.31112142])

In [16]:
error=[0 for j in range(250)]
for i in range(250):
    print('Epoch: ',i)
    mean,varience=train_continuous(train_image_number,i)
    error[i]=test_continuous(mean,varience,test_image_number)

Epoch:  0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Epoch:  10
Epoch:  11
Epoch:  12
Epoch:  13
Epoch:  14
Epoch:  15
Epoch:  16
Epoch:  17
Epoch:  18
Epoch:  19
Epoch:  20
Epoch:  21
Epoch:  22
Epoch:  23
Epoch:  24
Epoch:  25
Epoch:  26
Epoch:  27
Epoch:  28
Epoch:  29
Epoch:  30
Epoch:  31
Epoch:  32
Epoch:  33
Epoch:  34
Epoch:  35
Epoch:  36
Epoch:  37
Epoch:  38
Epoch:  39
Epoch:  40
Epoch:  41
Epoch:  42
Epoch:  43
Epoch:  44
Epoch:  45
Epoch:  46
Epoch:  47
Epoch:  48
Epoch:  49
Epoch:  50
Epoch:  51
Epoch:  52
Epoch:  53
Epoch:  54
Epoch:  55
Epoch:  56
Epoch:  57
Epoch:  58
Epoch:  59
Epoch:  60
Epoch:  61
Epoch:  62
Epoch:  63
Epoch:  64
Epoch:  65
Epoch:  66
Epoch:  67
Epoch:  68
Epoch:  69
Epoch:  70
Epoch:  71
Epoch:  72
Epoch:  73
Epoch:  74
Epoch:  75
Epoch:  76
Epoch:  77
Epoch:  78
Epoch:  79
Epoch:  80
Epoch:  81
Epoch:  82
Epoch:  83
Epoch:  84
Epoch:  85
Epoch:  86
Epoch:  87
Epoch:  88
Epoch:  89
Epoch:  90
Epoch:  91
Epoch:  

In [2]:
x=range(250)
plt.plot(x,error,'b')
plt.show()

NameError: name 'plt' is not defined

In [19]:
np.argmin(error)

45

In [20]:
error[45]

0.1976